In [1]:
#!pip install transformers
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
#!tar -xzvf pytorch_weights.tar.gz
#!mv config.json pytorch/.
#!mv vocab.txt pytorch/.

In [2]:
#!pip install torch torchvision

In [1]:
import torch
from transformers import BertForMaskedLM, BertTokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("pytorch/")
#model.eval()

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = "[CLS] La [MASK] es el problema de nuestro [MASK] [SEP]"
masked_indxs = (2,8)

tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens])

predictions = model(tokens_tensor)[0]

for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
    print('MASK',i,':',predicted_token)

MASK 0 : ['ignorancia', 'guerra', 'pobreza', 'muerte', 'violencia']
MASK 1 : ['país', 'tiempo', 'pueblo', 'mundo', 'planeta']


In [4]:
import glob
import pickle
import sys

import numpy as np
import pandas as pd

from constants import POS_TAGGED_FOLDER


def get_tagged_sentences(folder):
    # Load all the tagged sentences included in the .pickle files 
    parsed_sentences = []
    for filename in glob.glob(folder + '*.pickle'):
        with open(filename, 'rb') as tagged_file:
            parsed_sentences = parsed_sentences + pickle.load(tagged_file, encoding="bytes")
    return parsed_sentences

tagged_sentences = get_tagged_sentences(POS_TAGGED_FOLDER)
data = [{'sentence': item[b'sentence'].decode('utf8').lower(), 'target': item[b'classification'].decode('utf8')} for item in tagged_sentences]
df = pd.DataFrame(data)

In [5]:
data[0]

{'sentence': '\ufeffdiscurso de la presidenta cristina fernández de kirchner en la inauguracion del 133° periodo de sesiones ordinarias del congreso nacional',
 'target': 'non-fact-checkable'}

In [6]:
df.head()

,sentence,target
0,﻿discurso de la presidenta cristina fernández ...,non-fact-checkable
1,muy buenos días a todos y todas,non-fact-checkable
2,vengo una vez más a cumplir con lo dispuesto p...,non-fact-checkable
3,y la verdad que quiero comenzar dando cuenta d...,non-fact-checkable
4,el día viernes 27 pude leer un tuit en la cuen...,non-fact-checkable


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
'''
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['target']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

train_texts = strat_train_set.drop(['target'], axis=1).astype(str)
train_labels = strat_train_set['target'].values.tolist()

test_texts = strat_test_set.drop(['target'], axis=1).values.tolist()
test_labels = strat_test_set['target'].values.tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)
'''
###
X = [e['sentence'] for e in data]
y = [0  if e['target'] == 'fact-checkable' else 1 for e in data]

sentences = ["[CLS] " + e['sentence'] + " [SEP]" for e in data]
labels = [0  if e['target'] == 'fact-checkable' else 1 for e in data]

train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=.2)

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [8]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [57]:
test_texts[:5]

['hay más de 5 millones de trabajadores que no están registrados',
 'las actitudes oportunistas nos han impedido consensuar una reforma una y otra vez',
 'no puede ser que los hogares más pudientes gasten 20 veces más gas que los humildes',
 'esa franja de 18 a 24 años en todo al mundo es la de mayor desocupación',
 'de las terminales radicadas en la argentina de los 468 mil autos que importaban solamente 46 mil autos son alcanzados el 10 por ciento']

In [9]:
import torch

class ChqDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ChqDataset(train_encodings, train_labels)
val_dataset = ChqDataset(val_encodings, val_labels)
test_dataset = ChqDataset(test_encodings, test_labels)

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [14]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = BertForSequenceClassification.from_pretrained("pytorch/")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pytorch/

Step,Training Loss
10,0.672505
20,0.680028
30,0.626214
40,0.606749
50,0.527369
60,0.509940
70,0.471546
80,0.390945
90,0.394037
100,0.354776


TrainOutput(global_step=620, training_loss=0.2276120831889491)

In [15]:
trainer.save_model('qsch')

In [16]:
trainer.evaluate()

{'eval_loss': 0.3881931006908417,
 'eval_accuracy': 0.919093851132686,
 'eval_f1': 0.9336870026525198,
 'eval_precision': 0.9263157894736842,
 'eval_recall': 0.9411764705882353,
 'epoch': 4.0}

In [102]:
demo_texts = ['hay más de 25 millones de trabajadores que están registrados', 
              'hay más de 50 millones de trabajadores que están registrados', 
              'discurso de la presidenta cristina fernández de kirchner en la inauguracion del 133° periodo',
             'vamos a otorgar un subsidio de 15 mil pesos por cada nuevo empleo registrado durante doce meses',
             'se le saca a la Ciudad el 12% del presupuesto']
demo_labels = [0, 0, 1, 1, 0]

In [103]:
demo_encodings = tokenizer(demo_texts, truncation=True, padding=True)

In [104]:
len(demo_encodings.values())

3

In [105]:
demo_dataset = ChqDataset(demo_encodings, demo_labels)

In [106]:
trainer.predict(demo_dataset)

PredictionOutput(predictions=array([[ 3.3871017, -3.7961414],
       [ 3.3651884, -3.7773092],
       [-3.2432277,  3.7316933],
       [-1.8270873,  3.0422065],
       [ 3.815342 , -4.118779 ]], dtype=float32), label_ids=array([0, 0, 1, 1, 0]), metrics={'eval_loss': 0.0020983389113098383, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0})

In [107]:
demo_texts

['hay más de 25 millones de trabajadores que están registrados',
 'hay más de 50 millones de trabajadores que están registrados',
 'discurso de la presidenta cristina fernández de kirchner en la inauguracion del 133° periodo',
 'vamos a otorgar un subsidio de 15 mil pesos por cada nuevo empleo registrado durante doce meses',
 'se le saca a la Ciudad el 12% del presupuesto']

In [117]:
#without labels
class ChqInferenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, total_items):
        self.encodings = encodings
        self.total_items = total_items

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #item['labels'] = torch.tensor(self.labels[idx])
        print(item)
        return item

    def __len__(self):
        return self.total_items

In [120]:
demo_encodings = tokenizer(demo_texts, truncation=True, padding=True)
nolabel_dataset = ChqInferenceDataset(demo_encodings, len(demo_texts))
output = trainer.predict(nolabel_dataset)

{'input_ids': tensor([    4,  1423,  1216,  1008,  2329,  2439,  1008,  3426,  1038,  1690,
        15462,     5,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([    4,  1423,  1216,  1008,  2916,  2439,  1008,  3426,  1038,  1690,
        15462,     5,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}
{'input_ids': tensor([    4,  8500,  1008,  1030, 17635,  6187,  1532,  8409, 30935,  9577,
         1008,   982,  1191,  1124,  3972,  1036,  1030,  7912,  4392,  2243,
         1072,

In [121]:
output.predictions

array([[ 3.3871017, -3.7961414],
       [ 3.3651884, -3.7773092],
       [-3.2432277,  3.7316933],
       [-1.8270873,  3.0422065],
       [ 3.815342 , -4.118779 ]], dtype=float32)